# Importacion de librerias

In [14]:
from bokeh.io import output_notebook,show
from bokeh.plotting import figure
from bokeh.layouts import layout
import pandas as pd
import numpy as np
import datetime as dt
import Local_funtion as Lf
from bokeh.sampledata.glucose import data

# Cleaning & preparing data

## Cleaning data

In [15]:
#Is important to know what kind of data we have
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52281 entries, 2010-03-24 09:51:00 to 2010-10-10 23:57:00
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   isig     52281 non-null  object
 1   glucose  52281 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.2+ MB


In [16]:
# For good practice is correct copy the data set, for modify the copy, and don´t touch the original data_set
data_glucose=data.copy()

#We have a problem, the problem is that we have one column, with string, and we want float data
#For this reason we create a function that can covert isig column in float data.
def convert_in_number(data_value):
    try: 
        data_value=float(data_value)
        return data_value
    except:
        data_value=0
        return data_value
data_glucose['isig_value']=data_glucose['isig'].apply(convert_in_number)

In [17]:
# reseteando el indice para poder aplicar las funcion apply scalar
data_glucose=data_glucose.reset_index()
data_glucose.head()

,datetime,isig,glucose,isig_value
0,2010-03-24 09:51:00,22.59,258,22.59
1,2010-03-24 09:56:00,22.52,260,22.52
2,2010-03-24 10:01:00,22.23,258,22.23
3,2010-03-24 10:06:00,21.56,254,21.56
4,2010-03-24 10:11:00,20.79,246,20.79


In [18]:
#Desglosando el HXH.
data_hxh=Lf.apply_scalars(data_glucose,timestamp_col='datetime',hxh=True)


In [19]:
#Reseteando la data, ahora con las columnas que necesitamos para el analisis
data_hxh=data_hxh.set_index('datetime')
data_hxh['DATE']=pd.to_datetime(data_hxh['DATE'])
data_hxh.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 52281 entries, 2010-03-24 09:51:00 to 2010-10-10 23:57:00
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   isig        52281 non-null  object        
 1   glucose     52281 non-null  int64         
 2   isig_value  52281 non-null  float64       
 3   HXH         52281 non-null  object        
 4   SHIFT       52281 non-null  object        
 5   DATE        52281 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 2.8+ MB


In [20]:
#Agrupando a la tabla por semana
#De este primer pedazo, queremos saber 4 cosas
#Distribución de los datos.
#Grafica de todos los datos con el valor de la glucosa
#Grafica de todos los datos con el valor de isig_value
#Grafica de unicamente la media por dia.

WEEK_DATE=data_hxh.loc['2010-04-06':'2010-05-03']

WEEK_DATE_GROUP_GLUCOSE=WEEK_DATE.groupby(['DATE'])['glucose'].mean().reset_index()
WEEK_DATE_GROUP_GLUCOSE=WEEK_DATE_GROUP_GLUCOSE.set_index('DATE')

WEEK_DATE_GROUP_ISIG=WEEK_DATE.groupby(['DATE'])['isig_value'].mean().reset_index()
WEEK_DATE_GROUP_ISIG=WEEK_DATE_GROUP_ISIG.set_index('DATE')

## Creando filtros para una mejor visualización

In [21]:
#WEEK_DATE.head()
WEEK_DATE_BIGGER_THAN_125=WEEK_DATE_GROUP_GLUCOSE.loc[(WEEK_DATE_GROUP_GLUCOSE['glucose']>=125)]
WEEK_DATE_LESS_THAN_125=WEEK_DATE_GROUP_GLUCOSE.loc[(WEEK_DATE_GROUP_GLUCOSE['glucose']<125)]

In [22]:
#WEEK_DATE.head()
WEEK_DATE_BIGGER_THAN_12=WEEK_DATE_GROUP_ISIG.loc[(WEEK_DATE_GROUP_ISIG['isig_value']>=12)]
WEEK_DATE_LESS_THAN_12=WEEK_DATE_GROUP_ISIG.loc[(WEEK_DATE_GROUP_ISIG['isig_value']<12)]

## Filtro de WEEK DAY

In [23]:
WEEK_DATE_PER_DAY_BIGGER_THAN_125=WEEK_DATE.loc[(WEEK_DATE['glucose']>=125)]
WEEK_DATE_PER_DAY_LESS_THAN_125=WEEK_DATE.loc[(WEEK_DATE['glucose']<125)]

# Generación de gráficas

## Histogram in bokeh

In [24]:
# Generando un histograma 1
figure_h1=figure(title='Histogram of level of glucose',plot_width=800,plot_height=400)
hist,edges=np.histogram(WEEK_DATE.glucose,density=True,bins=20)
figure_h1.quad(top=hist,bottom=0,left=edges[:-1],right=edges[1:],
      fill_color='#008080',line_color='black',legend_label='Glucose')

# Generando un histograma 2
figure_h2=figure(title='Histogram of level of isig',plot_width=800,plot_height=400)
hist,edges=np.histogram(WEEK_DATE.isig_value,density=True,bins=20)
figure_h2.quad(top=hist,bottom=0,left=edges[:-1],right=edges[1:],
      fill_color='#008080',line_color='black',legend_label='isig')

GlyphRenderer(id='2857', ...)

## Normal Graphs in Bokeh

In [25]:
# bloque de codigo 1
figure_1=figure(x_axis_type='datetime',title='level glucose by day',plot_width=800,plot_height=400)
figure_1.xgrid.grid_line_color=None
figure_1.ygrid.grid_line_alpha=0.7
figure_1.xaxis.axis_label='time'
figure_1.yaxis.axis_label='leavel glucose'
figure_1.vbar(x=WEEK_DATE_PER_DAY_BIGGER_THAN_125.index,top=WEEK_DATE_PER_DAY_BIGGER_THAN_125.glucose,color='#FF0000',width=0.9)
figure_1.vbar(x=WEEK_DATE_PER_DAY_LESS_THAN_125.index,top=WEEK_DATE_PER_DAY_LESS_THAN_125.glucose,color='#008000',width=0.9)
# bloque de codigo 2
figure_2=figure(x_axis_type='datetime',title='isig value by day',plot_width=800,plot_height=400)
figure_2.xgrid.grid_line_color=None
figure_2.ygrid.grid_line_alpha=0.7
figure_2.xaxis.axis_label='time'
figure_2.yaxis.axis_label='leavel isig'
figure_2.line(WEEK_DATE.index,WEEK_DATE.isig_value,color='#000080',line_width=1.5)

# bloque de codigo 3
figure_3=figure(x_axis_type='datetime',title='level glucose by date mean',plot_width=800,plot_height=400)
figure_3.xgrid.grid_line_color=None
figure_3.ygrid.grid_line_alpha=0.7
figure_3.xaxis.axis_label='time'
figure_3.yaxis.axis_label='leavel glucose'
figure_3.circle(WEEK_DATE_BIGGER_THAN_125.index,WEEK_DATE_BIGGER_THAN_125.glucose,
               size=10,line_color='navy',fill_color='#FF0000',fill_alpha=0.9)
figure_3.circle(WEEK_DATE_LESS_THAN_125.index,WEEK_DATE_LESS_THAN_125.glucose,
               size=10,line_color='navy',fill_color='#008000',fill_alpha=0.9)

# bloque de codigo 2
figure_4=figure(x_axis_type='datetime',title='isig value by day mean',plot_width=800,plot_height=400)
figure_4.xgrid.grid_line_color=None
figure_4.ygrid.grid_line_alpha=0.7
figure_4.xaxis.axis_label='time'
figure_4.yaxis.axis_label='leavel isig'
figure_4.circle(WEEK_DATE_BIGGER_THAN_12.index,WEEK_DATE_BIGGER_THAN_12.isig_value,
               size=10,line_color='navy',fill_color='#FF0000',fill_alpha=0.9)
figure_4.circle(WEEK_DATE_LESS_THAN_12.index,WEEK_DATE_LESS_THAN_12.isig_value,
               size=10,line_color='navy',fill_color='#008000',fill_alpha=0.9)




GlyphRenderer(id='3045', ...)

# Mostrando resultado

In [26]:
lay_out_project=layout([[figure_h1,figure_h2],[figure_1,figure_2],[figure_3,figure_4]])
show(lay_out_project)